In [1]:
# IMPORT ALL YOUR LIBRARIES
# SUGGESTED LIBRARIES
!pip install accelerate
!pip install transformers
import accelerate
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import TensorDataset
from transformers import TrainingArguments, Trainer
import matplotlib.pyplot as plt

In [2]:
!mkdir /content/data

In [3]:
result_df = pd.read_csv("/content/big_data_2.csv")
result_df

Contextual Content        label  \
0    The Davos/UK dominated media, obsessed with ma...   Antagonist   
1    2. The United States has announced another sig...  Protagonist   
2    itarian organization that the former vice pres...   Antagonist   
3    ound the world, but they are also turning to p...  Protagonist   
4    The sheer size and complexity of the archives ...  Protagonist   
..                                                 ...          ...   
436  America, blinded by its own hubris, is playing...  Protagonist   
437  Pence calls viral clip suggesting he cares mor...   Antagonist   
438  The US, with its reckless disregard for intern...   Antagonist   
439  change for bailout loans.\n\n“No bank should b...     Innocent   
440   & Views (@RMXnews) March 13, 2024\n\nShe find...   Antagonist   

             Mentioned Entity  Protagonist  Antagonist  Innocent  \
0    Davos/UK dominated media            0           1         0   
1                     Ukraine            1           0         0   
2              Vladimir Putin            0           1         0   
3               working class            1           0         0   
4                     Britain            1           0         0   
..                        ...          ...         ...       ...   
436                   America            1           0         0   
437                Mike Pence            0           1         0   
438                        US            0           1         0   
439                    Brazil            0           0         1   
440                   Ukraine            0           1         0   

                                                Prompt  
0    Given the context: The Davos/UK dominated medi...  
1    Given the context: 2. The United States has an...  
2    Given the context: itarian organization that t...  
3    Given the context: ound the world, but they ar...  
4    Given the context: The sheer size and complexi...  
..                                                 ...  
436  Given the context: America, blinded by its own...  
437  Given the context: Pence calls viral clip sugg...  
438  Given the context: The US, with its reckless d...  
439  Given the context: change for bailout loans.\n...  
440  Given the context:  & Views (@RMXnews) March 1...  

[441 rows x 7 columns]

In [4]:
label_df = result_df[['Prompt','Protagonist','Antagonist','Innocent']]
label_df

Prompt  Protagonist  \
0    Given the context: The Davos/UK dominated medi...            0   
1    Given the context: 2. The United States has an...            1   
2    Given the context: itarian organization that t...            0   
3    Given the context: ound the world, but they ar...            1   
4    Given the context: The sheer size and complexi...            1   
..                                                 ...          ...   
436  Given the context: America, blinded by its own...            1   
437  Given the context: Pence calls viral clip sugg...            0   
438  Given the context: The US, with its reckless d...            0   
439  Given the context: change for bailout loans.\n...            0   
440  Given the context:  & Views (@RMXnews) March 1...            0   

     Antagonist  Innocent  
0             1         0  
1             0         0  
2             1         0  
3             0         0  
4             0         0  
..          ...       ...  
436           0         0  
437           1         0  
438           1         0  
439           0         1  
440           1         0  

[441 rows x 4 columns]

In [5]:
train_df = label_df[:350]
validation_df = label_df[350:420]
test_df = label_df[420:]
train_labels_df = label_df[:350]
validation_labels_df = label_df[350:420]
test_labels_df = label_df[420:]


In [6]:
train_sentences = train_df['Prompt'].values.tolist()
train_labels = train_labels_df.iloc[:, 1:].values
train_labels = train_labels.astype(np.float32)

validation_sentences = validation_df['Prompt'].values.tolist()
validation_labels = validation_labels_df.iloc[:, 1:].values
validation_labels = validation_labels.astype(np.float32)

In [7]:
# from transformers import AutoTokenizer

# checkpoint = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=3, problem_type="multi_label_classification")
# model

from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3, problem_type="multi_label_classification")

model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_fe

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=".",                           # Output directory
    evaluation_strategy="epoch",              # Evaluate at the end of each epoch
    logging_dir="./logs",                     # Directory for logging
    save_strategy="epoch",                    # Save model at the end of each epoch
    num_train_epochs=10,                       # Reduce epochs for faster convergence
    per_device_train_batch_size=8,           # Increase batch size (if memory allows) for faster learning
    per_device_eval_batch_size=4,            # Increase eval batch size accordingly
    load_best_model_at_end=True,              # Load the best model based on eval metric
    weight_decay=0.01,                        # Regularization
    learning_rate=2e-5,                       # Increase learning rate for faster convergence
    warmup_steps=50,                         # More warm-up steps to help model stabilize
    logging_steps=10,                         # Log more frequently to monitor progress
    metric_for_best_model="f1"                # Choose F1 score as the metric to monitor
)


# training_args = TrainingArguments(
#     output_dir=".",
#     evaluation_strategy="epoch",
#     logging_dir="./logs",
#     save_strategy="epoch",
#     num_train_epochs = 8,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     load_best_model_at_end=True,
#     weight_decay=0.01,
#     learning_rate = 8e-6,
#     warmup_steps = 30,
#     logging_steps= 100,
#     metric_for_best_model="f1"
# )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## [20 Points] Task 04 -  Training Loop


In this task, you are required to implement the training loop for fine tuning your model. You are also required to plot on the same graph: Loss vs Epochs & Accuracy vs Epochs

In [9]:
class ArgDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [10]:
test_labels_df

Prompt  Protagonist  \
420  Given the context: The World Bank's  report un...            1   
421  Given the context: Washington's involvement in...            1   
422  Given the context: S/TRANS news agency.\n\nThi...            0   
423  Given the context: decried the decision, citin...            1   
424  Given the context: ce chief vows \n\n RAMSTEIN...            0   
425  Given the context: Sir Richard Kemp's assessme...            0   
426  Given the context: Yulia Navalnaya, the wife o...            1   
427  Given the context: cessarily determine events ...            1   
428  Given the context: The thought of my family an...            1   
429  Given the context: er we will have not strateg...            1   
430  Given the context: te, warned, “A third world ...            0   
431  Given the context: the early hours on February...            1   
432  Given the context: The School of International...            1   
433  Given the context: ce added that Meta's action...            0   
434  Given the context:  regard." He issued the wor...            0   
435  Given the context:  where the government on Tu...            1   
436  Given the context: America, blinded by its own...            1   
437  Given the context: Pence calls viral clip sugg...            0   
438  Given the context: The US, with its reckless d...            0   
439  Given the context: change for bailout loans.\n...            0   
440  Given the context:  & Views (@RMXnews) March 1...            0   

     Antagonist  Innocent  
420           0         0  
421           0         0  
422           1         0  
423           0         0  
424           1         0  
425           0         1  
426           0         0  
427           0         0  
428           0         0  
429           0         0  
430           0         1  
431           0         0  
432           0         0  
433           1         0  
434           1         0  
435           0         0  
436           0         0  
437           1         0  
438           1         0  
439           0         1  
440           1         0

In [11]:
train_encodings = tokenizer(train_sentences, padding = True, truncation = True, max_length = 512)
train_dataset = ArgDataset(train_encodings, train_labels)

validation_encodings = tokenizer(validation_sentences, padding = True, truncation = True, max_length = 512)
validation_dataset = ArgDataset(validation_encodings, validation_labels)

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

label_names = train_labels_df.columns[1:].tolist()

def compute_metrics(pred):
    labels = pred.label_ids
    preds = (pred.predictions > 0.5).astype(int)

    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)

    # Generate classification report
    # report = classification_report(labels, preds, target_names=label_names, zero_division=0)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        # 'classification_report': report
    }

import wandb
wandb.init(mode="disabled")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


In [ ]:
log_history

In [ ]:
import matplotlib.pyplot as plt

# Extract the log history from the trainer
log_history = trainer.state.log_history

# Initialize lists for storing values
losses = []
accuracies = []
precisions = []
f1_scores = []
recalls = []

# Parse through the log history and extract values
for entry in log_history:
    if 'eval_loss' in entry and 'eval_accuracy' in entry and 'eval_precision' in entry and 'eval_recall' in entry and 'eval_f1' in entry:
        losses.append(entry['eval_loss'])
        accuracies.append(entry['eval_accuracy'])
        precisions.append(entry['eval_precision'])
        f1_scores.append(entry['eval_f1'])
        recalls.append(entry['eval_recall'])

# Define epochs (adjust based on your actual epoch range)
epochs = list(range(1, len(losses) + 1))

# Create a figure and axis for plotting
fig, ax1 = plt.subplots(figsize=(14, 6))

# Plot the first metric (loss) on the primary y-axis
ax1.plot(epochs, losses, color='tab:orange', label='Validation Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Validation Loss', color='tab:orange')
ax1.tick_params(axis='y', labelcolor='tab:orange')

# Create a secondary y-axis for Accuracy
ax2 = ax1.twinx()
ax2.plot(epochs, accuracies, color='tab:blue', linestyle='--', label='Accuracy')
ax2.set_ylabel('Accuracy', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

# Create a tertiary y-axis for Precision
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 60))  # Offset the third y-axis
ax3.plot(epochs, precisions, color='tab:green', linestyle='-', label='Precision')
ax3.set_ylabel('Precision', color='tab:green')
ax3.tick_params(axis='y', labelcolor='tab:green')

# Create a fourth y-axis for Recall
ax4 = ax1.twinx()
ax4.spines['right'].set_position(('outward', 120))  # Offset the fourth y-axis
ax4.plot(epochs, recalls, color='tab:red', linestyle=':', label='Recall')
ax4.set_ylabel('Recall', color='tab:red')
ax4.tick_params(axis='y', labelcolor='tab:red')

# Create a fifth y-axis for F1-score
ax5 = ax1.twinx()
ax5.spines['right'].set_position(('outward', 180))  # Offset the fifth y-axis
ax5.plot(epochs, f1_scores, color='tab:purple', linestyle='-.', label='F1-Score')
ax5.set_ylabel('F1-Score', color='tab:purple')
ax5.tick_params(axis='y', labelcolor='tab:purple')

# Add legends for each line
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()
lines4, labels4 = ax4.get_legend_handles_labels()
lines5, labels5 = ax5.get_legend_handles_labels()

ax2.legend(lines1 + lines2 + lines3 + lines4 + lines5,
           labels1 + labels2 + labels3 + labels4 + labels5,
           loc='lower left', bbox_to_anchor=(1.05, 1))

# Title and grid
plt.title('Validation Metrics vs Epochs')
plt.grid(False)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
model.save_pretrained("/content/")
model = BertForSequenceClassification.from_pretrained("/content/")

In [ ]:
# test_df.insert(0, 'Index', range(1, len(test_labels_df) + 1))
test_labels_df

In [ ]:
# test_labels_df.insert(0, 'Index', range(1, len(test_labels_df) + 1))
test_labels_df

In [ ]:
predictions = []
threshold = 0.4

for idx, row in test_df.iterrows():
    arg_id = row['Index']
    text = row['Prompt']
    encoding = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**encoding)

    sigmoid = torch.nn.Sigmoid()
    predictions_tensor = sigmoid(outputs.logits)

    predicted_labels = []
    for prob in predictions_tensor.flatten():
        if prob >= threshold:
          label = 1
        else:
          label =  0
        predicted_labels.append(label)

    predictions.append([arg_id] + predicted_labels)

data = {'Index': [entry[0] for entry in predictions]}

for i, label in enumerate(test_labels_df.columns[2:]):
    data[label] = [entry[i+1] for entry in predictions]
predictions_df = pd.DataFrame(data)


# output_path = "/content/output/my_model_predictions.tsv"
# predictions_df.to_csv(output_path, sep='\t', index=False)


In [ ]:
predictions_df

In [ ]:
true_pred = test_labels_df[['Index','Protagonist','Antagonist','Innocent']]
true_pred

In [ ]:
true_labels = true_pred[['Protagonist', 'Antagonist', 'Innocent']]
pred_labels = predictions_df[['Protagonist', 'Antagonist', 'Innocent']]

f1_average = f1_score(true_labels, pred_labels, average='macro')
print(f"Average F1 Score: {f1_average}")